In [ ]:
GRID = []
with open("day10_input.txt") as file:
    for row, line in enumerate(file):
        GRID.append(line.strip())
        if (col := line.find("S")) != -1:
            START = (row, col)

In [ ]:
def find_neighbours(pos):
    row, col = pos
    if not (0 <= row < len(GRID) and 0 <= col < len(GRID[0])):
        return []

    match GRID[row][col]:
        case "|":
            return [(row - 1, col), (row + 1, col)]
        case "-":
            return [(row, col - 1), (row, col + 1)]
        case "L":
            return [(row - 1, col), (row, col + 1)]
        case "J":
            return [(row - 1, col), (row, col - 1)]
        case "7":
            return [(row + 1, col), (row, col - 1)]
        case "F":
            return [(row + 1, col), (row, col + 1)]
        case "S":
            candidates = []
            neighbours = [
                (row + dx, col + dy) for dx, dy in zip((0, 0, 1, -1), (1, -1, 0, 0))
            ]
            for neighbour in neighbours:
                if pos in find_neighbours(neighbour):
                    candidates.append(neighbour)
            return candidates
        case _:
            return []

# Part 1


In [ ]:
def find_distances(start):
    """Use BFS to find the distance to all nodes from start."""
    visited = {start: 0}
    queue = [start]
    while queue:
        node = queue.pop(0)
        for neighbour in find_neighbours(node):
            if neighbour not in visited:
                visited[neighbour] = visited[node] + 1
                queue.append(neighbour)

    return visited


print("Answer:", max(find_distances(START).values()))

# Part 2


Use Dan Sundays winding number algoritm:
https://en.wikipedia.org/wiki/Point_in_polygon#Winding_number_algorithm

1. We need to follow along the loop in one direction until we're back at the start.
2. For each point, we need to know whether we're moving "up" or "down" relative to the previous point.


In [ ]:
def create_polygon(pos):
    """Use DFS to follow the polygon and create a list of nodes."""
    # Nudge the search in one direction around the loop
    first_neighbour = find_neighbours(pos)[0]
    # Use a dict for O(1) lookup
    visited = {pos: None, first_neighbour: None}
    queue = [pos, first_neighbour]
    while queue:
        node = queue.pop()
        for neighbour in find_neighbours(node):
            if neighbour not in visited:
                visited[neighbour] = None
                queue.append(neighbour)

    polygon = list(visited.keys())
    polygon.append(pos)  # Close the polygon
    return polygon


polygon = create_polygon(START)

In [ ]:
def edge_function(point, edge: tuple[tuple[int, int], tuple[int, int]]):
    """Juan Pineda's edge function.

    Returns negative if point is on the left side of edge, positive if on the right
    side, and 0 if on the edge."""
    y, x = point
    (y0, x0), (y1, x1) = edge
    return (x - x0) * (y1 - y0) - (y - y0) * (x1 - x0)

In [ ]:
def point_inside_polygon(point, polygon) -> bool:
    """Dan Sundays' winding number algorithm."""
    winding_number = 0
    for edge in zip(polygon, polygon[1:]):
        # if edge crosses ray upwards and point is strictly left of edge
        if (edge[0][1] <= point[1] < edge[1][1]) and edge_function(point, edge) < 0:
            winding_number += 1

        # if edge crosses ray downwards and point is strictly right of edge
        elif (edge[0][1] > point[1] >= edge[1][1]) and edge_function(point, edge) > 0:
            winding_number -= 1

    # If winding number is 0, then point is outside the polygon
    return winding_number != 0

In [ ]:
num_inside = 0
for row in range(len(GRID)):
    for col in range(len(GRID[0])):
        if (row, col) in polygon:
            continue
        if point_inside_polygon((row, col), polygon):
            num_inside += 1

print("Answer:", num_inside)

# Part 2: Alternative solution


1. Use the sholelace formula to calculate the area of the polygon. https://en.wikipedia.org/wiki/Shoelace_formula
2. Use Picks theorem to calculate the number of points inside the polygon. https://en.wikipedia.org/wiki/Pick%27s_theorem


In [ ]:
total = 0
for p1, p2 in zip(polygon, polygon[1:] + [polygon[0]]):
    total += p1[0] * p2[1] - p1[1] * p2[0]

# If the polygon order is clockwise, the total will be negative
area = abs(total) / 2
area

In [ ]:
# Pick's theorem:
# Area = num_points_inside + len(polygon)/2 - 1

print("Answer:", area - (len(polygon) - 1) / 2 + 1)